Failing to load Falcon3B (or the original for that matter), but loading other dataset works.ipynb

In [1]:
#%pip uninstall -y bitsandbytes
#%pip install bitsandbytes
#%pip install -U transformers accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 29.6 MB/s eta 0:00:00


In [1]:
%pip install -q datasets trl peft bitsandbytes sentencepiece wandb datasets

In [2]:
import os
import gc
import torch

import transformers
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, BitsAndBytesConfig
from datasets import load_dataset
from peft import LoraConfig, PeftModel, get_peft_model, prepare_model_for_kbit_training
from trl import DPOTrainer
import bitsandbytes as bnb
from google.colab import userdata
import wandb

model_name = "tiiuae/Falcon3-7B-Base"
new_model = "DPO-Falcon-3B-Base"

# Define BitsAndBytes configuration
#bnb_config = BitsAndBytesConfig(
#    load_in_4bit=True,
#    bnb_4bit_compute_dtype=torch.float16,
#    bnb_4bit_quant_type="nf4",
#    bnb_4bit_use_double_quant=True,
#)

# Define BitsAndBytes configuration for 8-bit quantization
bnb_config = BitsAndBytesConfig(
    load_in_8bit=True,
    llm_int8_threshold=6.0,
    llm_int8_has_fp16_weight=True,
)


def format_data(example):
    # Falcon doesn't use a chat template by default, so we'll create a simple format
    system = f"System: {example['system']}\n" if example['system'] else ""
    prompt = f"{system}User: {example['question']}\nAssistant: "

    # Format chosen and rejected answers
    chosen = example['chosen']
    rejected = example['rejected']

    return {
        "prompt": prompt,
        "chosen": chosen,
        "rejected": rejected,
    }




Trying to load alternative dataset

In [3]:


# Install TensorFlow Datasets (if not already installed)
!pip install tensorflow-datasets

# Import necessary libraries
import tensorflow_datasets as tfds
import tensorflow as tf

# Load the ai2_arc_with_ir dataset
dataset_name = "ai2_arc_with_ir"
print(f"Loading dataset: {dataset_name}")

# Load the dataset with the default configuration
dataset, info = tfds.load(dataset_name, with_info=True, as_supervised=False)

# Print dataset info
print(info)

# Explore dataset splits
print("Dataset splits:", list(dataset.keys()))

# Access a specific split (e.g., 'train')
train_data = dataset['train']

# Display some examples
print("\nSample examples from the 'train' split:")
for example in train_data.take(5):
    print(example)


Loading dataset: ai2_arc_with_ir


Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/3 [00:00<?, ? splits/s]

...

Generating validation examples...:   0%|          | 0/299 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/ai2_arc_with_ir/ARC-Challenge-IR/incomplete.RUEW0N_1.0.0/ai2_arc_with_ir-v…

Generating test examples...:   0%|          | 0/1172 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/ai2_arc_with_ir/ARC-Challenge-IR/incomplete.RUEW0N_1.0.0/ai2_arc_with_ir-t…

Dataset ai2_arc_with_ir downloaded and prepared to /root/tensorflow_datasets/ai2_arc_with_ir/ARC-Challenge-IR/1.0.0. Subsequent calls will reuse this data.
tfds.core.DatasetInfo(
    name='ai2_arc_with_ir',
    full_name='ai2_arc_with_ir/ARC-Challenge-IR/1.0.0',
    description="""
    A new dataset of 7,787 genuine grade-school level, multiple-choice science
    questions, assembled to encourage research in advanced question-answering. The
    dataset is partitioned into a Challenge Set and an Easy Set, where the former
    contains only questions answered incorrectly by both a retrieval-based algorithm
    and a word co-occurrence algorithm. We are also including a corpus of over 14
    million science sentences relevant to the task, and an implementation of three
    neural baseline models for this dataset. We pose ARC as a challenge to the
    community.
    
    Compared to the original dataset, this adds context sentences obtained through
    information retrieval in the same way

In [7]:
print(data_list[0])


{'answerKey': 2, 'choices': {'label': array([0, 1, 2, 3]), 'text': array([b'are low-density gases.', b'are not abundant on Earth.',
       b'have complete outermost energy levels.',
       b'have two electrons in their outermost energy level.'],
      dtype=object)}, 'id': b'Mercury_401011', 'paragraph': b'It and the other noble gases - helium, neon, krypton, xenon, and radon - will react with other substances only under extreme conditions. The noble gases The noble, or inert, gases are helium, neon, argon, krypton, xenon and radon. The rare gases are helium, neon, argon, krypton or xenon. The noble "gases" are helium, neon, argon, krypton and xenon. The noble gases are helium, neon, argon, krypton, xenon and radon. These occur for the noble gases helium, neon, argon, krypton, xenon, radon and ununoctium. Because of their chemical inertness, the elements in this group are called the Nobel Gases : Helium Neon Argon Krypton Xenon Radon Group I Elements The elements in this group have one

In [8]:
import tensorflow_datasets as tfds
from datasets import Dataset
from transformers import AutoTokenizer
from peft import LoraConfig

# Load the TensorFlow dataset
tfds_dataset = tfds.load('ai2_arc_with_ir', split='train', as_supervised=False)

# Convert the TensorFlow dataset to a list of Python dictionaries
data_list = list(tfds.as_numpy(tfds_dataset))

# Convert the list of dictionaries into a Hugging Face Dataset
train_dataset = Dataset.from_list(data_list)

# Tokenizer
model_name = "gpt2"  # Replace with your model name
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "left"



Map:   0%|          | 0/1119 [00:00<?, ? examples/s]

ValueError: text input must be of type `str` (single example), `List[str]` (batch or single pretokenized example) or `List[List[str]]` (batch of pretokenized examples).

In [9]:
# # Define format_data function
# def format_data(example):
#     # Tokenize input and format as needed
#     tokenized = tokenizer(
#         example["question"],  # Replace 'question' with the correct key from your dataset
#         truncation=True,
#         padding="max_length",
#         max_length=512,
#     )
#     return tokenized

def format_data(example):
    # Decode bytes to string
    question_text = example["question"].decode("utf-8")
    paragraph_text = example["paragraph"].decode("utf-8")

    # If you also want to include choices:
    choices_list = [c.decode("utf-8") for c in example["choices"]["text"]]

    # Combine them into one prompt or handle them separately
    # Example: question + paragraph + choices
    combined_text = (
        f"Paragraph: {paragraph_text}\n\n"
        f"Question: {question_text}\n\n"
        f"Choices: " + ", ".join(choices_list)
    )

    tokenized = tokenizer(
        combined_text,
        truncation=True,
        padding="max_length",
        max_length=512,
    )
    return tokenized



# Map the formatting function to the dataset
formatted_dataset = train_dataset.map(
    format_data,
    remove_columns=train_dataset.column_names
)

# LoRA configuration
peft_config = LoraConfig(
    r=16,
    lora_alpha=16,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=['query_key_value', 'dense', 'dense_h_to_4h', 'dense_4h_to_h']
)

# Print a sample from the formatted dataset
print(formatted_dataset[0])


Map:   0%|          | 0/1119 [00:00<?, ? examples/s]

{'input_ids': [50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256

In [4]:




# # Load dataset
# #dataset = load_dataset("Intel/orca_dpo_pairs")['train']
# #original_columns = dataset.column_names

# # Tokenizer
# tokenizer = AutoTokenizer.from_pretrained(model_name)
# tokenizer.pad_token = tokenizer.eos_token
# tokenizer.padding_side = "left"

# # Format dataset
# dataset = dataset.map(
#     format_data,
#     remove_columns=original_columns
# )

# # LoRA configuration
# peft_config = LoraConfig(
#     r=16,
#     lora_alpha=16,
#     lora_dropout=0.05,
#     bias="none",
#     task_type="CAUSAL_LM",
#     target_modules=['query_key_value', 'dense', 'dense_h_to_4h', 'dense_4h_to_h']
# )



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


AttributeError: 'dict' object has no attribute 'map'

In [10]:
# Model to fine-tune - now with quantization config
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    torch_dtype=torch.float16,
    trust_remote_code=True
)
model.config.use_cache = False
model = prepare_model_for_kbit_training(model)

`low_cpu_mem_usage` was None, now default to True since model is quantized.


model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

AttributeError: 'Tensor' object has no attribute 'SCB'

In [ ]:

# Reference model - now with quantization config
ref_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    torch_dtype=torch.float16,
    trust_remote_code=True
)

# Training arguments
training_args = TrainingArguments(
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    gradient_checkpointing=True,
    learning_rate=5e-5,
    lr_scheduler_type="cosine",
    max_steps=200,
    save_strategy="no",
    logging_steps=1,
    output_dir=new_model,
    optim="paged_adamw_32bit",
    warmup_steps=100,
    bf16=True,
    report_to="wandb",
)

# Create DPO trainer
dpo_trainer = DPOTrainer(
    model,
    ref_model,
    args=training_args,
    train_dataset=dataset,
    tokenizer=tokenizer,
    peft_config=peft_config,
    beta=0.1,
    max_prompt_length=1024,
    max_length=1536,
)

# Fine-tune model with DPO
dpo_trainer.train()

# Save artifacts
dpo_trainer.model.save_pretrained("final_checkpoint")
tokenizer.save_pretrained("final_checkpoint")

# Flush memory
del dpo_trainer, model, ref_model
gc.collect()
torch.cuda.empty_cache()

# Reload model in FP16
base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    return_dict=True,
    torch_dtype=torch.float16,
    trust_remote_code=True
)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Merge base model with the adapter
model = PeftModel.from_pretrained(base_model, "final_checkpoint")
model = model.merge_and_unload()

# Save model and tokenizer
model.save_pretrained(new_model)
tokenizer.save_pretrained(new_model)

# Push to HF Hub (if token is configured)
if 'hf_token' in locals():
    model.push_to_hub(new_model, use_temp_dir=False, token='hf_token')
    tokenizer.push_to_hub(new_model, use_temp_dir=False, token='hf_token')
else:
  hf_token = ''

# Test generation
test_prompt = "User: What is a Large Language Model?\nAssistant: "

# Create pipeline
pipeline = transformers.pipeline(
    "text-generation",
    model=new_model,
    tokenizer=tokenizer,
    trust_remote_code=True
)

# Generate text
sequences = pipeline(
    test_prompt,
    do_sample=True,
    temperature=0.7,
    top_p=0.9,
    num_return_sequences=1,
    max_length=200,
)
print(sequences[0]['generated_text'])

In [ ]:

# Model to fine-tune - now with quantization config
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    torch_dtype=torch.float16,
    trust_remote_code=True
)
model.config.use_cache = False
model = prepare_model_for_kbit_training(model)

# Reference model - now with quantization config
ref_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    torch_dtype=torch.float16,
    trust_remote_code=True
)

# Training arguments
training_args = TrainingArguments(
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    gradient_checkpointing=True,
    learning_rate=5e-5,
    lr_scheduler_type="cosine",
    max_steps=200,
    save_strategy="no",
    logging_steps=1,
    output_dir=new_model,
    optim="paged_adamw_32bit",
    warmup_steps=100,
    bf16=True,
    report_to="wandb",
)

# Create DPO trainer
dpo_trainer = DPOTrainer(
    model,
    ref_model,
    args=training_args,
    train_dataset=dataset,
    tokenizer=tokenizer,
    peft_config=peft_config,
    beta=0.1,
    max_prompt_length=1024,
    max_length=1536,
)

# Fine-tune model with DPO
dpo_trainer.train()

# Save artifacts
dpo_trainer.model.save_pretrained("final_checkpoint")
tokenizer.save_pretrained("final_checkpoint")

# Flush memory
del dpo_trainer, model, ref_model
gc.collect()
torch.cuda.empty_cache()

# Reload model in FP16
base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    return_dict=True,
    torch_dtype=torch.float16,
    trust_remote_code=True
)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Merge base model with the adapter
model = PeftModel.from_pretrained(base_model, "final_checkpoint")
model = model.merge_and_unload()

# Save model and tokenizer
model.save_pretrained(new_model)
tokenizer.save_pretrained(new_model)

# Push to HF Hub (if token is configured)
if 'hf_token' in locals():
    model.push_to_hub(new_model, use_temp_dir=False, token=hf_token)
    tokenizer.push_to_hub(new_model, use_temp_dir=False, token=hf_token)
#else:
#  hf_token = ''

# Test generation
test_prompt = "User: What is a Large Language Model?\nAssistant: "

# Create pipeline
pipeline = transformers.pipeline(
    "text-generation",
    model=new_model,
    tokenizer=tokenizer,
    trust_remote_code=True
)

# Generate text
sequences = pipeline(
    test_prompt,
    do_sample=True,
    temperature=0.7,
    top_p=0.9,
    num_return_sequences=1,
    max_length=200,
)
print(sequences[0]['generated_text'])

In [ ]:
#slightly different try, same result

import os
import gc
import torch

import transformers
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments
from datasets import load_dataset
from peft import LoraConfig, PeftModel, get_peft_model, prepare_model_for_kbit_training
from trl import DPOTrainer
import bitsandbytes as bnb
from google.colab import userdata
import wandb

model_name = "tiiuae/Falcon3-7B-Base"
new_model = "DPO-Falcon-3B-Base"

def format_data(example):
    system = f"System: {example['system']}\n" if example['system'] else ""
    prompt = f"{system}User: {example['question']}\nAssistant: "

    return {
        "prompt": prompt,
        "chosen": example['chosen'],
        "rejected": example['rejected'],
    }

# Load dataset
dataset = load_dataset("Intel/orca_dpo_pairs")['train']
original_columns = dataset.column_names

# Tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "left"

# Format dataset
dataset = dataset.map(
    format_data,
    remove_columns=original_columns
)

# LoRA configuration
peft_config = LoraConfig(
    r=16,
    lora_alpha=16,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=['query_key_value', 'dense', 'dense_h_to_4h', 'dense_4h_to_h']
)

# Model to fine-tune with basic Int8 quantization
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    load_in_8bit=True,
    device_map="auto",
    trust_remote_code=True
)
model.config.use_cache = False
model = prepare_model_for_kbit_training(model)

# Reference model with basic Int8 quantization
ref_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    load_in_8bit=True,
    device_map="auto",
    trust_remote_code=True
)

# Training arguments
training_args = TrainingArguments(
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    gradient_checkpointing=True,
    learning_rate=5e-5,
    lr_scheduler_type="cosine",
    max_steps=200,
    save_strategy="no",
    logging_steps=1,
    output_dir=new_model,
    optim="paged_adamw_32bit",
    warmup_steps=100,
    bf16=True,
    report_to="wandb",
)

# Create DPO trainer
dpo_trainer = DPOTrainer(
    model,
    ref_model,
    args=training_args,
    train_dataset=dataset,
    tokenizer=tokenizer,
    peft_config=peft_config,
    beta=0.1,
    max_prompt_length=1024,
    max_length=1536,
)

# Fine-tune model with DPO
dpo_trainer.train()

# Save artifacts
dpo_trainer.model.save_pretrained("final_checkpoint")
tokenizer.save_pretrained("final_checkpoint")

# Flush memory
del dpo_trainer, model, ref_model
gc.collect()
torch.cuda.empty_cache()

# Reload model
base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    trust_remote_code=True
)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Merge base model with the adapter
model = PeftModel.from_pretrained(base_model, "final_checkpoint")
model = model.merge_and_unload()

# Save model and tokenizer
model.save_pretrained(new_model)
tokenizer.save_pretrained(new_model)

# Push to HF Hub (if token is configured)
if 'hf_token' in locals():
    model.push_to_hub(new_model, use_temp_dir=False, token=hf_token)
    tokenizer.push_to_hub(new_model, use_temp_dir=False, token=hf_token)

# Test generation
test_prompt = "User: What is a Large Language Model?\nAssistant: "

# Create pipeline
pipeline = transformers.pipeline(
    "text-generation",
    model=new_model,
    tokenizer=tokenizer,
    trust_remote_code=True
)

# Generate text
sequences = pipeline(
    test_prompt,
    do_sample=True,
    temperature=0.7,
    top_p=0.9,
    num_return_sequences=1,
    max_length=200,
)
print(sequences[0]['generated_text'])